# Lab 06 - Bias-Variance Trade-off

In passed labs discussed different regression and classification algorithms. For many of these algorithms we saw rather than a single model, we are in fact able to fit a family of models:
- In the case of polynomial fitting, we can fit different models by changing the polynomial degree $k$. When we increased $k$ out models became more elaborate and were able to fit closer to the data.
- In the case of decision trees, we can determine the maximum tree depth. Deeper trees form a higher resolution of partitioning of the feature space, enabling finer classification.
- In the case of the $k$-NN classifier, prediction is made based on a neighborhood of size $k$. 

For all of these examples, we built the intuition that in order to fit a good model for our traning data we desire a more complex model. However, we also saw that more complex models suffer from high losses over new test datasets. To formalize these two opposing forces we defined the following:

- The **bias** of an estimator $\widehat{\theta}$ of some parameter $\theta$ is its expected deviation from the true value: $Bias\left(\widehat{\theta}\right) = \mathbb{E}\left[\widehat{\theta}\right] - \theta$.
- The **variance** of an estimator $\widehat{\theta}$ of some parameter $\theta$ is the expected values of the squared sampling deviations: $var\left(\widehat{\theta}\right)=\mathbb{E}\left[\left(\widehat{\theta}-\mathbb{E}\left[\widehat{\theta}\right]\right)^2\right]$.


Then we saw that we can decompose the generalization eror of a learner into two parts: the bias and the variance:

$$ L_\mathcal{D} \left(h_S\right) = \underset{bias}{\underbrace{L_\mathcal{D}\left(h^*\right)}} + \underset{variance}{\underbrace{L_\mathcal{D}\left(h_S\right) - L_\mathcal{D}\left(h^*\right)}} $$ 

where $h^* = argmin_{h\in\mathcal{H}} L_\mathcal{D}\left(h\right)$ and $h_S=\mathcal{A}\left(S\right)$, for $S$ the training set. Specifically when the loss function is the MSE function, the generalization loss is decomposted as:

$$ \mathbb{E}\left[\left(\widehat{y}-y^*\right)^2\right] = \mathbb{E}\left[\left(\widehat{y}-\overline{y}\right)^2\right] + \left(\overline{y}-y^*\right)^2 = var\left(\widehat{y}\right) + Bias^2\left(\widehat{y}\right) $$

where $\overline{y} = \mathbb{E}\left[\widehat{y}\right]$.


In the following lab we will visualize the bias-variance trade-off in their influence on a classifier's decision boundary as well as construct a bias-variance graph over some dataset.

In [7]:
import sys 
sys.path.append("../")
from utils import *

In [8]:
from scipy.stats import norm
from sklearn.model_selection import train_test_split
from sklearn.neighbors import KNeighborsClassifier
from sklearn.tree import DecisionTreeClassifier, plot_tree, export_text

np.random.seed(1)

## Data Generation

For the purpose of this lab, let us define $\mathbb{R}^2$ the feature space where a the class of a data-point is $1$ if it is in one of three rectangles. Otherside it has a class of $0$. To form the different datasets that are used below we generate a grid of points in $\mathbb{R}^2$ which are labeled by the `true_labels` function.

In [9]:
def true_labels(X):
    return np.array([1 if ((1 <= x[0] <= 4.25 and 1 <= x[1] <= 5.25)  or 
                           (6 <= x[0] <= 9 and 4 <= x[1] <= 9)  or 
                           (.75 <= x[0] <= 1.75 and 8.1 <= x[1] <= 9.4))
                     else 0 for x in X])

surface_range = [-.5,10.5], [-.5,10.5]
xx, yy = np.meshgrid(np.linspace(0, 10, 25), np.linspace(0, 10, 25))
X = np.c_[xx.ravel(), yy.ravel()]
y_ = true_labels(X)

go.Figure([decision_surface(true_labels, *surface_range), 
           go.Scatter(x = X[:, 0], y=X[:, 1], mode = 'markers', marker=dict(color=y_, colorscale=custom), showlegend=False)],
          layout=go.Layout(title=r"$\text{(1) True Data Layout}$")).show()

## Simulate Noisy Samples

We can sample points from the grid above to generate a dataset. In order to model noise in the labels we do the following: for each sample we perform a Bernulli experiment (for some probability $p$). Then depending on the results we either leave the sample's label as is or replace it. The dataset (in avarage) will have $m\cdot p$ points that their label is incorrect.
*Implementation note: Binomial distribution with n=1 is Bernoulli distribution.*

In Figure 2 the backround surface denotes the true label of the data. The points are the training sample with their color representing their class. Notice that red points on a blue surface or blue points on a red surface are precisely the samples where their label in the train set is wrong.

In [10]:
def add_noise(y, p=.1):
    return np.array([y, 1-y]).T[range(len(y)), np.random.binomial(1, p, len(y))]

train_idx, noise_level = np.random.choice(range(len(X)), 200, replace=False), .25

X_train, y_train = X[train_idx], y_[train_idx]
y_noisy = add_noise(y_train, noise_level)

go.Figure([decision_surface(true_labels, *surface_range),
           go.Scatter(x = X_train[:,0], y=X_train[:, 1], mode = 'markers', marker = dict(color=y_noisy, colorscale=custom))],
         layout= go.Layout(title=rf"$\text{{(2) True Data Layout With Noisy Sample - }}p={noise_level}$", xaxis_title=r"$x_1$", yaxis_title=r"$x_2$"))

## Decision boundaries

To visualize a classifier's prediction let us plot its decision boundaries over the feature space $\mathbb{R}^2$. The learner we will be using is of decision trees. Recall that a decision tree is a partitioning of the feature space into a disjoint union of "boxes". Then, the prediction made for a new sample $\mathbf{x}\in\mathbb{R}^2$ is by the majoiry vote of the "box" in falls into. Insead of only looking at the $argmax$ label of a box, let us visualize the proportion of positives out of all data-points in the box. So: $$ prediction\left(B\right) = \frac{\left|\left\{x\in B | class(x) = 1\right\}\right|}{\left|B\right|} $$

This provides us with a continuous scale in the range of $\left[0,1\right]$ with the meaning of "how likely is this box positive". Below we visualize the decision boundaries of decision tree classifiers of increasing depth.
- Run the following block with `visualize_noisy_data = False` to see decision boundaries of the taken sample but with its true labels.
- Then run the following block with `visualize_noisy_data = True` to see decision boundaries of the taken sample but with the added noise.

In [11]:
def classifier(model_type="tree", **kwargs):
    if model_type == 'knn':
        return KNeighborsClassifier(kwargs['k'])
    # if model_type == "tree":
    #     return DecisionTreeClassifier(**kwargs)

In [12]:
visualize_noisy_data = True

frames = []
for k in range(1, 15):
    x, y, title = (X_train, y_noisy, "Noisy Sample") if visualize_noisy_data else (X_train, y_train, "True Layout")
    
    clf = classifier("knn", max_depth=k).fit(x, y)
    frames.append(go.Frame(data = [decision_surface(lambda x: clf.predict_proba(x)[:,1], *surface_range),
                                   go.Scatter(x = x[:,0], y=x[:, 1], mode = 'markers', 
                                              marker = dict(color=y, colorscale=custom))],
                           traces=[0],
                           layout = go.Layout(title=rf"$\text{{(3) Decision Boundary Of {title} - Max Depth }}k={k}$")))

go.Figure(data = frames[0]["data"], frames=frames[1:], 
          layout = go.Layout(title=frames[0]["layout"]["title"],
                             updatemenus=[dict(type="buttons", 
                                               buttons=[AnimationButtons.play(), AnimationButtons.pause()])])).show()

KeyError: 'k'

Notice that when fitting the model over the true data layout, the algorithm converges into a specific tree of depth $8$. Once it reaches this depth, the tree does not change even when increasing the `max_depth`. At this point it has reaches a training error of zero. However, when running over the noisy sample, the fitted trees continue to change until a much deeper depth. At the end it achieves a training error of zero, but the outputted model is very complex. We expect this model to perform poorly over a new test set.

# Computing Bias and Variance Of Model

As the fitted model (of any learning algorithm) is returned over a given **specific** and **noisy** sample, this model is in fact a random variable. As such also its predictions are **random variables**.

Let us calculate the bias and variance properties of a decision tree of increasing depth. Recall the bias and variance definitions:
$$ Bias\left(\widehat{\theta}\right) = \mathbb{E}\left[\widehat{\theta}\right] - \theta $$
$$ var\left(\widehat{\theta}\right)=\mathbb{E}\left[\left(\widehat{\theta}-\mathbb{E}\left[\widehat{\theta}\right]\right)^2\right] $$

In both of these definitions, the expectation is over the selection of training sets $S\sim\mathcal{D}^m$. For each such set we produce an estimator $\widehat{\theta}$, which we then use to make the predictions.

So to calculate these measures we will take the training sampled used in figure 2. We will refer to `X_train, y_train` as the ground truth. Then, we change the labels vector `y_train` to add noise and fit a decision tree of a given depth. We will repeat this process multiple times to aquire many estimator $\widehat{\theta}_1,\ldots,\widehat{\theta}_T$. Averaging over these estimators will provide us with the empirical expectation needed for both the bias and variance calculations.


In [ ]:
frames, ks = [], [1, 2, 8, 16]
for i in range(20):
    x, y, data = X_train, add_noise(y_train, 0.25), []
    for k in ks:
        clf = classifier("knn", max_depth=k).fit(x, y)
        data.extend([
            decision_surface( lambda x: clf.predict_proba(x)[:,1], *surface_range, showscale=False, dotted=False, density=40),
            go.Scatter(x = x[:,0], y=x[:, 1], mode = 'markers', marker = dict(color=y, colorscale=custom), showlegend=False)])
        
    frames.append(go.Frame(data = data, name=i,
                           traces = list(range(len(data))),#list(range(0, len(data), 2)),
                           layout = go.Layout(title=rf"$\text{{(4) Decision Boundaries - Iteration {i+1}}}$")))


fig = make_subplots(rows=2, cols=2, subplot_titles=[rf"$k={k}$" for k in ks])
fig.add_traces(data=frames[0]["data"], rows=[1,1,1,1,2,2,2,2], cols=[1,1,2,2,1,1,2,2])
fig.update(frames=frames)
fig.update_layout(title=r"$\text{(4) Decision Boundaries}$",updatemenus=[dict(type="buttons", buttons=[AnimationButtons.play(), AnimationButtons.pause()])])
fig.show()

What can we learn from figure 4? We begin with understanding the setup presented in the figure. 
- In all panels and animaiton frames we ovserve, the same data-points shown in figure 2. These stay constant across the figure.
- Between each animation frame the data-points are given a new label with probability $p=0.25$. This represents the noisiness of the data. All figure panels use the same noisy data in a given animation frame.
- In each panel a decision tree of different depth is fitted, and is represented by its decision boundary.



As each animation frame represents fitting a model over a dataset where the $X$ remains the same and the $y$ changes by a bit we are able to **observe** the bias and variance of the predictors.
- Notice that the models in the top row are **too simple**. They are not able to capture the complexity of the data. The hypothesis clesses used in each of the two top panels are: decision trees of depth at most 1 (left; i.e. decision stumps) and 2. There are simply no good eoungh hypotheses **in the hypothesis classes** to match the data. The best estimator the algorithm can find is still "far" from the true parameters. So the **bias** observed in the top panels is **high**.


- In the bottom row the hypothesis classes are sufficiently large to include trees of the correct depth for this data. As such the **bias** of the estimators retrieved by the learning algorithm in the bottom row is **low**. 


- Next, consider how consistent a prediction is on average. In the top row, thoguh the decision doundaries move between the different frames, each data-points recieves mostly the same prediction. This change in predicted labels is a proxy to the estimator's variance. As the predictions are mostly the same, the estimator's **variance** is **low**.


- Now, when we look at the bottom row we notice that in each frame the decision boundaries change dramatically. Each time they capture small different subsets of the data. Each time the estimated decision tree "manages" to correctly classify more of the data-points, even though the labels of each point might change between the animation frames. This means that the estimators seen in the different frames are very different from one another. As such, the **variance** of the estimators is **high**.

# Bias-Variance Trade-off

Lastly, after visualizing the effects of the bias and variance thoguh the decision doundaries of the classifiers, let us calculate the bias and variance measurements of the estimator and plot the bias-variance tradeoff graph. To calculate these measures over a given dataset we will calculate the empirical bias and variances of the estimator. We begin with computing the $Bias^2$, $Variance$ and $MSE$ of a given model: a decision tree classifier of depth at most $k=8$.

*Note that we are using the estimator's prediction of the probability being classified as $1$ and not the actual $0$ or $1$ classification prediction. We do so in order to use the squared loss function and not the misclassification loss function.*

In [ ]:
def calc_variance(y_hats, y_true):
    avg = np.mean(y_hats, axis=0)
    point_var = np.var(y_hats - avg, axis=0, ddof=1)
    var = np.mean(point_var)
    return var


def calc_bias(y_hats, y_true):
    E_y = y_hats.mean(axis=0)
    bias_sq = np.mean((E_y - y_true)**2)
    return bias_sq


def calc_MSE(y_hats, y_true):
    mse = np.mean((y_hats - y_true)**2)
    return mse


def run_simulations(model, X, y, X_test, y_test, p, repetitions):
    preds = np.zeros((repetitions, len(y_test)))
    for i in range(repetitions):
        # Fit a model over noised training set
        clf = model.fit(X, add_noise(y, p))
        # Predict probabilty of classifying to class 1 over test set
        preds[i] = clf.predict_proba(X_test)[:, 1]

    return calc_bias(preds, y_test), calc_variance(preds, y_test), calc_MSE(preds, y_test)    


In [ ]:
p, repetitions = .25, 500

model = classifier('knn', max_depth=8)
bias, variance, MSE = run_simulations(model,  
                                      X[train_idx], y_[train_idx], 
                                      X[~train_idx], y_[~train_idx],
                                      p, repetitions)

print("Bias:\t\t\t", bias)
print("Variance:\t\t", variance)
print("Bias^2 + Variance:\t", bias + variance)
print("MSE:\t\t\t", MSE)

Now, let us repeat the process above for different values of $k$ to create the full bias-variance trade-off graph.

In [ ]:
def get_title_and_name(model_type):
    if model_type == "knn":
        return "K - Number of Neighbors", "K-NN"
    # if model_type == "tree":
    #     return "Tree Depth", "Desicion Tree"

In [ ]:
vals = list(range(1, 15))
x = list(range(len(vals)))
x_title, name = get_title_and_name('knn')

bias, variance, mse = np.zeros(len(vals)), np.zeros(len(vals)), np.zeros(len(vals))
for i, param in enumerate(vals):
    model = classifier("knn", max_depth=param)
    bias[i], variance[i], mse[i] = run_simulations(model, 
                                                   X[train_idx], y_[train_idx], 
                                                   X[~train_idx], y_[~train_idx], 
                                                   p, repetitions)


go.Figure([
    go.Scatter(x=x, y=variance, mode='markers + lines', name=r'$Variance$'),
    go.Scatter(x=x, y=bias, mode='markers + lines', name=r'$Bias^2$'),
    go.Scatter(x=x, y=mse, mode='markers + lines', name=r'$MSE$'),
    go.Scatter(x=x, y=np.array(bias) + np.array(variance), mode='markers + lines', name=r'$Bias^2 + Variance$')])\
.update_layout(title=rf"$\text{{(5) Bias-Variance Graph Of {name} }}$", 
               xaxis = dict(title=x_title, tickvals=x, ticktext=vals)).show()

# Time To Think...

In this lab we have seen a very important phenomina that is observed in many of the algorithms seen in the course, and that is also encountered in many real-life machine learning problems. Let us view the bias-variance trade-off also in the $k$-Nearest-Neighbors classifier. Make the following modification to the code above:

1. Add $k$-NN model to the `classifier` function:
```
    if model_type == 'knn':
        return KNeighborsClassifier(kwargs['k'])
```

2. Add the following to the `get_title_and_name` function:
```
    if model_type == "knn":
        return "K - Number of Neighbors", "K-NN"   
```
3. Change the block of code above to call the $k$-NN classifier instead of the decision tree classifier. In addition make sure that model complexity increases as we move to the right on the x-axis.
        
<br>After computing the bias-variance tradeoff graph:
1. Is there any meaningfull qualitative difference between the two classifiers ($k$-NN and decision boundaries)?
2. What is this difference and how can you explain it? 
3. How can you explain it?
